In [1]:
import wobble
import numpy as np
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

/Users/mbedell/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
o = 56
data = wobble.Data('51peg_e2ds.hdf5', filepath='data/', orders=[o])
results = wobble.Results(data)

In [19]:
data = wobble.Data('51peg_e2ds.hdf5', filepath='data/', orders=np.arange(72))
results = wobble.Results(data=data)

In [20]:
for r in range(data.R):
    model = wobble.Model(data, results, r)
    model.add_star('star')
    model.add_telluric('tellurics', rvs_fixed=True, variable_bases=3)
    print("--- ORDER {0} ---".format(r))
    wobble.optimize_order(model, niter=60)
    #if (r % 5) == 0:
    #    results.write('results_order{0}.hdf5'.format(data.orders[r]))

--- ORDER 0 ---


100%|██████████| 60/60 [00:01<00:00, 30.65it/s]


--- ORDER 1 ---


100%|██████████| 60/60 [00:02<00:00, 29.65it/s]


--- ORDER 2 ---


...

100%|██████████| 60/60 [00:04<00:00, 13.66it/s]


--- ORDER 70 ---


100%|██████████| 60/60 [00:04<00:00, 13.22it/s]


--- ORDER 71 ---


100%|██████████| 60/60 [00:04<00:00, 13.09it/s]


In [21]:
results.component_names

['star', 'tellurics']

In [22]:
results.write('test.hdf5')

Results: writing to test.hdf5


In [23]:
results.combine_orders('star')

AttributeError: 'Results' object has no attribute 'data'

In [ ]:
for r in range(data.R):
    model = wobble.Model(data, results, r)

    model.add_star('star')
    model.add_telluric('tellurics', rvs_fixed=True, variable_bases=3)

    model.setup()

    model.optimize(niter=60)


In [ ]:
results.write('test.hdf5')

In [ ]:
validation_epochs = np.random.choice(data.N, data.N//10, replace=False)
training_epochs = np.delete(np.arange(data.N), validation_epochs)
    
training_data = wobble.Data('51peg_e2ds.hdf5', filepath='data/', orders=[o], mask_epochs=validation_epochs)
training_results = wobble.Results(training_data)

validation_data = wobble.Data('51peg_e2ds.hdf5', filepath='data/', orders=[o], mask_epochs=training_epochs)
validation_results = wobble.Results(validation_data)

In [ ]:
r = 0
training_model = wobble.Model(training_data, training_results, r)
training_model.add_star('star')
training_model.add_telluric('tellurics', rvs_fixed=True, variable_bases=3)
training_model.setup()

validation_model = wobble.Model(validation_data, validation_results, r)
validation_model.add_star('star')
validation_model.add_telluric('tellurics', rvs_fixed=True, variable_bases=3)
validation_model.setup()

In [ ]:
regularization_dict = {training_model.components[0].L1_template_tensor: 1.e1,
                      training_model.components[0].L2_template_tensor: 1.e3,
                      training_model.components[1].L1_template_tensor: 1.e3,
                      training_model.components[1].L2_template_tensor: 1.e6,
                      training_model.components[1].L1_basis_vectors_tensor: 1.e8,
                      training_model.components[1].L2_basis_vectors_tensor: 1.e0,
                      training_model.components[1].L2_basis_weights_tensor: 1.0,}

In [ ]:
training_model.optimize(niter=60, feed_dict=regularization_dict)

In [ ]:
validation_dict = {**regularization_dict}
for c in validation_model.components:
    validation_dict[getattr(c, 'template_xs')] = getattr(training_model.results, c.name+'_template_xs')[r]
    validation_dict[getattr(c, 'template_ys')] = getattr(training_model.results, c.name+'_template_ys')[r]

In [ ]:
session = wobble.utils.get_session()
for i in tqdm(range(60)):
    session.run(validation_model.components[0].opt_rvs, feed_dict=validation_dict)
session.run(validation_model.nll)

In [ ]:
for c in validation_model.components:
    validation_model.results.update(c)

In [ ]:
n = validation_epochs[0]
plt.plot(data.xs[r][n], data.ys[r][n], color='k')
plt.plot(data.xs[r][n], validation_results.star_ys_predicted[r][n], color='r', alpha=0.5)
plt.plot(data.xs[r][n], validation_results.tellurics_ys_predicted[r][n], color='b', alpha=0.5)
#plt.xlim([8.252, 8.255])

In [ ]:
#results.write('results.hdf5')